In [2]:
#IPython magic to reload libraries
%load_ext autoreload
%autoreload 2

# Extra code so Juypter doesn't crash on mac os
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [40]:
import sys
sys.path.insert(0, 'utils')  # noqa

import numpy as np
import gym
import tasks
import ntm
import interfaces

In [88]:
# Training Params
train_iter = 1000
batch_size = 1


In [89]:
# Task
string_len=5
num_char = 5
env = tasks.Copy(string_len=string_len, max_char=num_char)



In [228]:
# Controller 
Controller = ntm.NTM(memory_vector_len=10, num_char=num_char, max_run_time=20)

In [229]:
# #Over Fit on single char
# output = Controller.run(tape_input=1, writes_left=1)
# print(output)
# Controller.update(output_target=4)

In [230]:
# # Test RL training
# output = Controller.run(tape_input=1, writes_left=1)
# print(Controller.run_time)
# print(Controller.run_time_left(1))
# Controller.update_store(1, done=True)


In [231]:
# print(Controller.run_time)

In [ ]:
output_control_cmds = ["no","yes"]
input_control_cmds = {-1:"left",0:"stay",1:"right"}
# input_control_cmds = ["left","stay","right"]
int_to_char = ["A","B","C","D","E","F","G"]
render_env = False
for i in range(train_iter):
# for i in range(1):

    # Collect experince using current policy/controller
    for j in range(batch_size):
        
        episode_steps = 0
        observation, target, done, info = env.reset()
        if render_env: env.render()

        while True:
            episode_steps += 1
            
            #Run RL-NTM controller and get outputs
            outputs = Controller.run(tape_input=observation, writes_left=info)

            output_content = outputs[0] # Which character to write (ignored if the above sub-action is 0) 
            output_control = outputs[1] #  Whether to write to the output tape ["no","yes"] = [0,1]
            input_control = outputs[2] # Direction to move the read head ["left","right"] = [0,1]
           
            actions = (input_control, output_control, output_content)
            
            if render_env:
                print("Policy - obs: {} action: {}, {}, {}".format(int_to_char[observation],
                      int_to_char[output_content],
                      output_control_cmds[output_control],
                      input_control_cmds[input_control]))

            # target is the expected output for the action that was just taken
            observation, target, done, info = env.step(actions)
            if render_env: env.render()
                
            if output_control_cmds[output_control] == "yes":
                Controller.update_store(output_target=target, done=done)
                
            if done:
                done = False
                break
        